In [1]:
# from PIL import Image
# temp_image = Image.open('datasets/FLIR/train/PreviewData/FLIR_00011.jpeg')
# print(temp_image.size)
# temp_image

In [ ]:
# %cd ../PreviewData


In [ ]:
# import json # import json module

# with open('datasets/FLIR/train/Annotations/FLIR_00014.json') as f:
#     json_data = json.load(f)

# #print(json_data['annotation'])

# for i in range(len(json_data['annotation'])):
#     print(json_data['annotation'][i]['category_id'])
#     print(json_data['annotation'][i]['bbox'])



In [ ]:
# from matplotlib import pyplot as plt
# import numpy
# plt.figure(figsize = (15,15))
# # tensor([[317.0266,  65.5659, 453.2461, 347.0377],
# #         [169.0079,  27.5498, 244.5284, 313.9996]

# temp_image = Image.open('datasets/FLIR/train/PreviewData/FLIR_00014.jpeg')
# plt.imshow(temp_image)
# box = [0,0,0,0]

# for i in range(len(json_data['annotation'])):
#    box[0], box[1],box[2], box[3] =  json_data['annotation'][i]['bbox']
#    box[2] +=box[0]
#    box[3]+= box[1]
#    x_values = [box[0], box[0], box[2], box[2], box[0]]
#    y_values = [box[1], box[3], box[3], box[1], box[1]]
#    plt.plot(x_values, y_values, linewidth = 3, label ='asdf')
#    #print(json_data['annotation'][i]['category_id'])

# # The_boxes = (prediction[0]['boxes']).tolist()
# # for box in The_boxes:
# #   x_values = [box[0], box[0], box[2], box[2], box[0]]
# #   y_values = [box[1], box[3], box[3], box[1], box[1]]
# #   plt.plot(x_values, y_values, linewidth = 3)

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import json

class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PreviewData"))))
        self.jsons = list(sorted(os.listdir(os.path.join(root, "Annotations"))))
        #self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        self.Kimg = []
    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PreviewData", self.imgs[idx])
        json_path = os.path.join(self.root, "Annotations", self.jsons[idx])
        #mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        with open(json_path) as f:
            json_data = json.load(f)
        

        boxes = []
        labels = []
        area = []
        if len(json_data['annotation']) <1:
            jpegSTR = str(json_data['image']['file_name']) + '.jpeg'
            jsonSTR = str(json_data['image']['file_name']) + '.json'
            
            !mv datasets/FLIR/train/PreviewData/{jpegSTR} datasets/FLIR/train/NotData/
            !mv datasets/FLIR/train/Annotations/{jsonSTR} datasets/FLIR/train/NotData/
            return str(json_data['image'])
        else:
            box = [0,0,0,0]
        
        
        for i in range(len(json_data['annotation'])):
            # true인 곳의 위치들이 array로 pos에 들어감
            # where : 괄호 안에 참값이 들어갈 때 그 index값을 반환해줌 , 즉 좌표를 반환해줌 pos[0]은 y값들의 위치 pos[1]은 x값들의 위치
            box[0], box[1],box[2], box[3] =  json_data['annotation'][i]['bbox']
            box[2] +=box[0]
            box[3]+= box[1]
            xmin = (box[0])
            xmax = (box[2])
            ymin = (box[1])
            ymax = (box[3])
            boxes.append([xmin, ymin, xmax, ymax])
            area.append((xmax - xmin) * (ymax-ymin))
            if (int(json_data['annotation'][i]['category_id']) > 4):
                labels.append(5)
            else:
                labels.append(int(json_data['annotation'][i]['category_id']))
            
            
            if not (int(json_data['annotation'][i]['category_id']) in self.Kimg):
                #print(int(json_data['annotation'][i]['category_id']))
                self.Kimg.append(int(json_data['annotation'][i]['category_id']))
        
        area = torch.as_tensor(area, dtype=torch.float32)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
#        print("i : ", i)
        #labels = labels[0]
        labels = torch.tensor((labels), dtype=torch.int64)
#        print(labels)
#        labels = torch.ones((len(labels),), dtype=torch.int64)

#        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        
     #   area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) # 그냥 가로 * 세로 로 면적 계산
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(json_data['annotation']),), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
 #       target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
dataset = PennFudanDataset('datasets/FLIR/train')
print(len(dataset))
# for i in ((dataset)):
#     i


In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # COCO에서 미리 학습된 인스턴스 분할 모델을 읽어옵니다.
    #model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # 분류를 위한 입력 특징 차원을 얻습니다.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    #in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    
    # hidden_layer = 256
    # and replace the mask predictor with a new one
    #model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

    
    return model


In [ ]:

# Download TorchVision repo to use some files from
# references/detection
# !git clone https://github.com/pytorch/vision.git
# %cd vision
# !git checkout v0.3.0

# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../

In [ ]:
#%%shell

# #Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# %cd cocoapi/PythonAPI
# !python setup.py build_ext install

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# use our dataset and defined transformations
dataset = PennFudanDataset('datasets/FLIR/train', get_transform(train=True))
dataset_test = PennFudanDataset('datasets/FLIR/train', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=2, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

In [ ]:
'datasets/FLIR/train'

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import torch.nn as nn
import os







#os.environ["CUDA_VISIBLE_DIVICES"] = '0, 1, 2, 3'

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

#device = torch.device("cuda:0,1,2,3" if torch.cuda.is_available() else "cpu")

# our dataset has two classes only - background and person
num_classes = 6

# get the model using our helper function





model = get_instance_segmentation_model(num_classes)
model.load_state_dict(torch.load("./checkpoints/epoch?????.pth"))





# if torch.cuda.device_count() >1:
#     print("Let's use ", torch.cuda.device_count(),"GPUs!")
#     model = nn.DataParallel(model)
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00005,
                             weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
epoch = 343

path = "./checkpoints/epoch" + str(epoch) + ".pth"
print("save : ", path)
torch.save(model.state_dict(), path)


In [ ]:
# let's train it for 10 epochs
num_epochs = 100

for epoch in range(30, num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=50)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)
    if(epoch % 4 == 0):
        path = "./checkpoints/epoch" + str(epoch) + ".pth"
        print("save : ", path)
        torch.save(model.state_dict(), path)
        
        

In [ ]:
dataset_test2 = PennFudanDataset('datasets/FLIR/train', get_transform(train=False))
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"


In [ ]:
#print(torch.cuda.device_count())
# model = get_instance_segmentation_model(num_classes)

# model = torch.nn.DataParallel(model, device_ids=[0])
# model.to(device)

# device_ = torch.device("cuda")

test_nums = 1


# pick one image from the test set
img, _ = dataset_test[1]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
#dataset_test[1]

In [ ]:
from matplotlib import pyplot as plt
import numpy
plt.figure(figsize = (15,15))
# tensor([[317.0266,  65.5659, 453.2461, 347.0377],
#         [169.0079,  27.5498, 244.5284, 313.9996]

plt.imshow(img.permute(1, 2, 0))

color_set = ['w','r','b','y','g','m']
The_boxes = (prediction[0]['boxes']).tolist()

this_labels = (prediction[0]['labels']).tolist()


for i, box in enumerate(The_boxes):
  x_values = [box[0], box[0], box[2], box[2], box[0]]
  y_values = [box[1], box[3], box[3], box[1], box[1]]
  plt.plot(x_values, y_values, linewidth = 3, color = color_set[this_labels[i]])

In [ ]:
from matplotlib import pyplot as plt
import numpy
plt.figure(figsize = (15,15))
# tensor([[317.0266,  65.5659, 453.2461, 347.0377],
#         [169.0079,  27.5498, 244.5284, 313.9996]

plt.imshow(img.permute(1, 2, 0))

color_set = ['w','r','b','y','g','m']
The_boxes = (dataset_test[test_nums][1]['boxes']).tolist()

this_labels = (dataset_test[test_nums][1]['labels']).tolist()


for i, box in enumerate(The_boxes):
  x_values = [box[0], box[0], box[2], box[2], box[0]]
  y_values = [box[1], box[3], box[3], box[1], box[1]]
  plt.plot(x_values, y_values, linewidth = 3, color = color_set[this_labels[i]])